In [9]:
from datetime import datetime
from time import sleep
import pickle
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
# waits
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import sqlalchemy as sa 
import os
from selenium.common.exceptions import NoSuchElementException

In [11]:
login = os.getenv('login')
password = os.getenv('password')
host = os.getenv('host')

Считываем заранее сскаченный csv файл со списком нужных запросов


In [14]:
def postgres_con():
    engine = sa.create_engine(f'postgresql://{login}:{password}@{host}/postgres') 
    con=engine.connect()
    return con

In [12]:
def baza_sql():
    con=postgres_con()
    query= "SELECT * FROM sravni.baza"
    baza=pd.read_sql(query, con)
    baza.drop('index', axis=1, inplace=True)
    return baza

In [8]:
df=pd.read_csv('baza_wordstat.csv')
baza = df.iloc[:, 0].apply(lambda x: re.sub(r';.*?;', ' ', x))

In [37]:
engine = sa.create_engine(f'postgresql://{login}:{password}@{host}/postgres') 
con=engine.connect()
baza.to_sql(con=con, schema='sravni',if_exists='append', name='baza',index=False)

398

In [15]:
baza=baza_sql()

Функции  загрузки cookies


In [17]:
def load_cookie(browser, path):
    with open(path, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            print(cookie)
            try:
                browser.add_cookie(cookie) 
            except Exception as e:
                continue

Запкускаем код с cookie чтобы обойти капчу

Начинаем сбор данных 

In [18]:
def parser (search_list):
    options = Options()
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--headless") 
    service=Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=service, options=options)
    browser.get('https://ya.ru/search/')
    load_cookie(browser, 'cookies_yan.dat')
    browser.refresh()
    
    queries_list=[]
    
    for d in search_list:    
        u = f'https://yandex.ru/search/?text={d}&rstr=-213&lr=213'
        print(u)
        browser.get(u)
    
        for i in browser.find_elements(By.CLASS_NAME, "serp-item"):
                
          organic__subtitle = i.find_elements(By.CLASS_NAME, 'organic__subtitle')
          serp_url__item = i.find_elements(By.CLASS_NAME, 'serp-url__item')
          
          if organic__subtitle:
            h = organic__subtitle[0].text
            t = 'Ads'

          elif serp_url__item:   
            h = serp_url__item[0].text
            t = 'Organic'
          else:
            continue
                    
          queries_list.append([d,h.split('›')[0],t])
          
    return queries_list

In [ ]:
domen_list = parser(baza['сравним ру;256820;'])


Преобразуем полученные данные в Dataframe

In [24]:
def create_dataFrame(data):
    con=postgres_con()
    df = pd.DataFrame(data, columns = ['Request','Domen', "Type"])
    df['Date'] = str(datetime.now())
    df.to_sql(con=con, schema='sravni',if_exists='append', name='sravni_queries')
    con.close()
    
    
    

In [25]:
def main():
    postgres_con()
    baza=baza_sql()
    domen_list=parser(baza['сравним ру;256820;'])
    create_dataFrame(domen_list)
   

In [26]:
main()

{'domain': '.yandex.ru', 'expiry': 1752489213, 'httpOnly': False, 'name': 'bh', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'EkEiR29vZ2xlIENocm9tZSI7dj0iMTI1IiwgIkNocm9taXVtIjt2PSIxMjUiLCAiTm90LkEvQnJhbmQiO3Y9IjI0IioCPzA6CSJXaW5kb3dzImD9iZazBg=='}
{'domain': '.yandex.ru', 'expiry': 1752489205, 'httpOnly': False, 'name': 'L', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'ARcEZ39jCXZVUnl/bmtAYH1aB3VHcn1vA19eJzVXUQ==.1717929204.15768.345607.20255b1f2d8b4112bacb703cb69518d8'}
{'domain': '.yandex.ru', 'expiry': 1752489205, 'httpOnly': True, 'name': 'sessionid2', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '3:1717929204.5.0.1717929204978:29v8bQ:47.1.2:1|1115296289.0.2.3:1717929204|3:10289651.616361.fakesign0000000000000000000'}
{'domain': '.yandex.ru', 'expiry': 1752489205, 'httpOnly': True, 'name': 'sessar', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '1.1190.CiAVuUxoTx2v32uTdDefnS8q_EAyBk-D04dgPOxa3nyicw.ixjx44hbbtFaDN_J9XlMyb3vRQHzM3NB6

Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sqlalchemy\pool\base.py", line 986, in _finalize_fairy
    fairy._reset(
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sqlalchemy\pool\base.py", line 1432, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sqlalchemy\engine\default.py", line 698, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



https://yandex.ru/search/?text=сравни ру авиабилеты &rstr=-213&lr=213
https://yandex.ru/search/?text=сравни ру страховка спортивная &rstr=-213&lr=213
https://yandex.ru/search/?text=сравни ру займ +на карту онлайн &rstr=-213&lr=213
https://yandex.ru/search/?text=сравни ру ипотека сбербанк &rstr=-213&lr=213
https://yandex.ru/search/?text=страховая компания сравни ру &rstr=-213&lr=213
https://yandex.ru/search/?text=сравни ру страхование жизни +для ипотеки &rstr=-213&lr=213
https://yandex.ru/search/?text=сравни учи ру ответы &rstr=-213&lr=213
https://yandex.ru/search/?text=страховка +на машину онлайн сравни ру &rstr=-213&lr=213
https://yandex.ru/search/?text=сравни ру обмен валюты &rstr=-213&lr=213
https://yandex.ru/search/?text=сделать страховку сравни ру &rstr=-213&lr=213
https://yandex.ru/search/?text=сравни ру альфа &rstr=-213&lr=213
https://yandex.ru/search/?text=рассчитать страховку онлайн сравни ру &rstr=-213&lr=213
https://yandex.ru/search/?text=сравни ру хорошие сравни ру &rstr=-2